In [93]:
import pandas as pd

base_path = './ml-100k/'

# 1. Load ratings (u.data: tab-separated userId | movieId | rating | timestamp)
ratings_cols = ['userId','movieId','rating','timestamp']
ratings = pd.read_csv(
    f'{base_path}u.data',
    sep='\t',
    names=ratings_cols,
    engine='python'
)

# 2. Load movies (u.item: pipe-separated; first 5 cols + 19 genre flags)
item_cols = ['movieId','title','release_date','video_release_date','IMDb_URL']
genre_cols = [
    'unknown','Action','Adventure','Animation',"Children's",'Comedy','Crime',
    'Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery',
    'Romance','Sci-Fi','Thriller','War','Western'
]
movies = pd.read_csv(
    f'{base_path}u.item',
    sep='|',
    names=item_cols + genre_cols,
    encoding='latin-1',
    engine='python'
)

# 3. Quick sanity-check
print(ratings.head())
print(movies.head())


   userId  movieId  rating  timestamp
0     196      242       3  881250949
1     186      302       3  891717742
2      22      377       1  878887116
3     244       51       2  880606923
4     166      346       1  886397596
   movieId              title release_date  video_release_date  \
0        1   Toy Story (1995)  01-Jan-1995                 NaN   
1        2   GoldenEye (1995)  01-Jan-1995                 NaN   
2        3  Four Rooms (1995)  01-Jan-1995                 NaN   
3        4  Get Shorty (1995)  01-Jan-1995                 NaN   
4        5     Copycat (1995)  01-Jan-1995                 NaN   

                                            IMDb_URL  unknown  Action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0       1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0       0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...        0       1   
4  http://

## Model Implementation

In [94]:
# mean rating of movies dataframe, converted to integer as to be a categorical feature
mean_ratings = ratings.groupby(ratings.movieId).mean()[['rating']].astype(int)

# merge movies dataframe on mean_ratings dataframe by movieId
# final_df contains genres as features (True or False) and Ratings as a feature (1-5)
final_df = pd.merge(mean_ratings, movies.set_index('movieId'), left_index= True, right_index=True)

def p_X_given_y(watched_df, row): #returns P(x_1 | y), P(x_2 | y), P(x_3 | y)...
    row = row.drop(['title', 'release_date', 'video_release_date', 'IMDb_URL']) #drop non-features
    matches = (watched_df[row.index] == row.values).sum() #count(X, y)
    return (1 + matches) / (1 + len(watched_df))  #P(X | y) = count(X, y) / count(y), with laplace smoothing


def p_y_given_X(row, watched_df, final_df): #P(y | X) ∝ P(Y) * ∏ P(X | y)
    p_y = len(watched_df) / len(final_df) #P(Y) = count(Y) / count(All)
    p_X_given_y_vector = p_X_given_y(watched_df, row) #P(x_1 | y), P(x_2 | y), P(x_3 | y)...
    return p_y * p_X_given_y_vector.prod() #P(y | X) ∝ P(Y) * ∏ P(x_i | y)


# compute P(y|X) for all unwatched movies
# accepts as input the whole dataframe of all movies(final_df), and the dataframe of all watched movies by the user(watched_df)
def naive_bayes(final_df, watched_df):
    unwatched_df = pd.concat([final_df, watched_df, watched_df]).drop_duplicates(keep=False) #unwatched_movies df
    unwatched_df['p_y_given_X'] = unwatched_df.apply(p_y_given_X, axis = 1, args = (watched_df, final_df)) # P(y | X) for all unwatched movies
    return unwatched_df.sort_values(by = 'p_y_given_X', ascending = False)['title'].head(10) # returns 10 movies with highest probability


naive_bayes(final_df, final_df.sample(frac=0.2))  # Randomly select 20% of the DataFrame as watched

movieId
1682         Scream of Stone (Schrei aus Stein) (1991)
627               Robin Hood: Prince of Thieves (1991)
1252                      Contempt (Mépris, Le) (1963)
1261                    Run of the Country, The (1995)
1264                            Nothing to Lose (1994)
1265                                  Star Maps (1997)
1266    Bread and Chocolate (Pane e cioccolata) (1973)
1267                                   Clockers (1995)
1268                                Bitter Moon (1992)
1558                                  Aparajito (1956)
Name: title, dtype: object

In [95]:
merged_df = ratings.merge(movies, on="movieId")
merged_df.sort_values(by = 'userId')#[merged_df['userId'] == 1]
#merged_df[merged_df['userId'] == 1]
hidden_df = merged_df[merged_df['userId'] == 1].sample(frac=0.2)
watched_df = merged_df[merged_df['userId'] == 1].drop(hidden_df.index)
result_df = naive_bayes(final_df, watched_df)


In [96]:
# import random
# #print(random.randint(3, 9))
# def validator(result_df, hidden_df):
#   value = 0
#   for i in result_df.index:
#     if i in hidden_df.index:
#       value+=1
#   percentage = value/len(result_df)
#   return percentage

# for i in range(10):
#   ID = random.randint(1, 943)
#   hidden_df = merged_df[merged_df['userId'] == ID].sample(frac=0.2)
#   watched_df = merged_df[merged_df['userId'] == ID].drop(hidden_df.index)
#   result_df = naive_bayes(final_df, watched_df)
#   print(validator(result_df, hidden_df))


In [97]:
movies_final = movies.drop(['title', 'release_date', 'video_release_date', 'IMDb_URL'], axis = 1).set_index('movieId')
user_rating = ratings[ratings['userId'] == 1]
user_rating_cleaned = user_rating.reset_index()[['movieId', 'rating']]
user_rating_cleaned['rating'] = user_rating_cleaned['rating'].isin([4,5])
user_rating_cleaned


,movieId,rating
0,61,True
1,189,False
2,33,True
3,160,True
4,20,True
...,...,...
267,28,True
268,172,True
269,122,False
270,152,True


In [98]:
movies_final = movies.drop(['title', 'release_date', 'video_release_date', 'IMDb_URL'], axis = 1).set_index('movieId')
user_rating = ratings[ratings['userId'] == 1]
user_rating_cleaned = user_rating.reset_index()[['movieId', 'rating']]
user_rating_cleaned['rating'] = user_rating_cleaned['rating'].isin([4,5])
user_rating_cleaned


def p_X_given_y(row, reviews): #returns P(x_1 | y), P(x_2 | y), P(x_3 | y)...
    matches = (reviews.values == row.values).sum(axis=0)
    return (1 + matches) / (1 + len(reviews))  #P(X | y) = count(X, y) / count(y), with laplace smoothing

def p_y_given_X(row, movies_final, ratings_final): #P(y | X) ∝ P(Y) * ∏ P(X | y)
    p_y = ratings_final['rating'].mean() #P(Y) = count(Y) / count(All)
    good_review_ids = ratings_final[ratings_final['rating']]['movieId']
    p_X_given_y_vector = p_X_given_y(row, movies_final.loc[good_review_ids]) #P(x_1 | y), P(x_2 | y), P(x_3 | y)...
    return p_y * p_X_given_y_vector.prod() #P(y | X) ∝ P(Y) * ∏ P(x_i | y)

def p_y_0_given_X(row, movies_final, ratings_final): #P(y | X) ∝ P(Y) * ∏ P(X | y)
    p_y = 1 - ratings_final['rating'].mean() #P(Y) = count(Y) / count(All)
    bad_review_ids = ratings_final[~ratings_final['rating']]['movieId']
    p_X_given_y_vector = p_X_given_y(row, movies_final.loc[bad_review_ids]) #P(x_1 | y), P(x_2 | y), P(x_3 | y)...
    return p_y * p_X_given_y_vector.prod() #P(y | X) ∝ P(Y) * ∏ P(x_i | y)

# compute P(y|X) for all unwatched movies
# accepts as input the whole dataframe of all movies(final_df), and the dataframe of all watched movies by the user(watched_df)
def naive_bayes(movies_final, user_rating_cleaned):
    unwatched_df = movies_final[~movies_final.index.isin(user_rating_cleaned['movieId'])].copy()
    p_y_given_X_col = unwatched_df.apply(p_y_given_X, axis = 1, args = (movies_final, user_rating_cleaned))
    p_y_0_given_X_col = unwatched_df.apply(p_y_0_given_X, axis = 1, args = (movies_final, user_rating_cleaned))
    unwatched_df['p_y_given_X'] = p_y_given_X_col
    unwatched_df['p_y_0_given_X'] = p_y_0_given_X_col
    unwatched_df['pred_rating'] = (unwatched_df['p_y_given_X'] > unwatched_df['p_y_0_given_X']).astype(int)
    return unwatched_df.sort_values(by = 'p_y_given_X', ascending = False)[['p_y_given_X', 'pred_rating']]
    
naive_bayes(movies_final, user_rating_cleaned)  # Randomly select 20% of the DataFrame as watched

,p_y_given_X,pred_rating
movieId,,
1682,4.878166e-02,1
1005,4.878166e-02,1
1564,4.878166e-02,1
985,4.878166e-02,1
981,4.878166e-02,1
...,...,...
993,1.783629e-07,0
426,1.502854e-07,0
560,7.570921e-08,0


In [99]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score

train_data, test_data = train_test_split(user_rating_cleaned, test_size=0.2)
train_predictions = naive_bayes(movies_final, train_data)
test_data_with_predictions = pd.merge(train_predictions, test_data, left_index = True, right_on = 'movieId')
test_data_with_predictions

y_true = test_data_with_predictions['rating']
y_pred = test_data_with_predictions['pred_rating']

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)

# Print the results
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")

Precision: 0.6341
Recall: 0.9286
Accuracy: 0.6909
